In [ ]:
import matplotlib.pyplot as plt

from pynq import Overlay
import numpy as np

from qsystem_2 import *
from qsystem2_asm import *

In [ ]:
outputChannel = 7
inputChannel = 0

soc = PfbSoc('qsystem_2.bit', force_init_clks=False)

In [ ]:
with ASM_Program() as p:
	#Write all memory addresses to program memory
	p.memri(0,2,2,"Encoded Frequency")
	p.memri(0,4,4,"Address")
	p.memri(0,5,5,"Gain")
	p.memri(0,6,6,"nsamp")
	p.memri(0,13,11,"tOffset")
	p.math(0,1,6,"+",13) #Add the pulse duration + the trigger offset to the device. 
	#Set up nsamp and DAC selection register.
	# For envelope, set outsel=00
	p.regwi(0,8,0b1000,"0b1000, stdysel = 1 (zero value), mode = 0 (nsamp), outsel = 00 (envelope).")
	p.bitwi(0,8,8, "<<", 16)
	p.bitw(0,6,6,"|",8)
	p.regwi(0,9,0xC001,"Trigger pmod0 bit0 and input channels 0 and 1") 
	#Delay the start a bit
	p.synci(1000)
	#Trigger value for the average block. 
	p.set(0,0,9,9,9,9,9,13)#Start the trigger at the offset value
	p.set(outputChannel,0,2,0,4,5,6,0)
	p.sync(0,1)
	#End the trigger signal
	p.seti(0,0,0,0)
	soc.tproc.load_asm_program(p)

In [ ]:
#Returns the time of flight for a pulse sent and received by the RF SoC device. All values sent and received are in clocks unless otherwise specified. 

def measureTOF(
        frequency = 500, 
        gain = 32767, 
        address = 0, 
        nsamp = 100, 
        tOffset = 0):

        #Write requsite values to the memory of the tproc
        freqRegDAC = freq2reg(soc.fs_dac, frequency, B=32)
        soc.tproc.single_write(addr=2, data = freqRegDAC)
        soc.tproc.single_write(addr=4, data = address)
        soc.tproc.single_write(addr=5, data = gain)
        soc.tproc.single_write(addr=6, data = nsamp)
        soc.tproc.single_write(addr=11, data = tOffset)

        # For envelope, upload envelope
        xg_i = gauss(mu=16*nsamp/2, si=nsamp, length=16*nsamp, maxv=30000)
        xg_q = np.zeros(len(xg_i))
        soc.gens[outputChannel-1].load(addr=address, xin_i=xg_i, xin_q=xg_q)

        #Set up the readout channel
        soc.readouts[inputChannel].set_out("product")
        soc.readouts[inputChannel].set_freq(frequency)
        decimatedLength = 1000
        soc.avg_bufs[inputChannel].config(address=0, length=1000)
        soc.avg_bufs[inputChannel].enable()

        #Restart the tproc
        soc.tproc.stop()
        soc.tproc.start()

        #Get the results of the run
        idec,qdec = soc.get_decimated(inputChannel, length = decimatedLength)
        
        amps = np.abs(idec + 1j*qdec)
        times = np.linspace(tOffset, tOffset+999, 1000)
        
        #Plot the amplitudes with the best guess line
        bestLine = np.where(amps == np.amax(amps))[0][0]
        bestLine = bestLine + tOffset
        
        fig = plt.figure()
        fig.set_figwidth(10)
        plt.plot(times, amps)
        plt.axvline(bestLine, ls="--", color="red", alpha=0.5, label="Detected Peak")
        
        tof = bestLine - (nsamp / 2) #Calculate where the pulse actually began
        
        plt.axvline(tof, ls="--", color="green", alpha = 0.5, label="Pulse Start")
        tofLabel = "Time of Flight: " + str(tof) + " clocks or " + str(tof * 2.6) + " ns"
        plt.axvspan(tOffset, tof, color="lightslategrey", alpha=0.2, label=tofLabel)
        plt.legend()
        plt.title("Time of Flight Test Readout")
        plt.show()
        

        #Return
        return tof

tof = measureTOF(tOffset = 0)
print("Time of flight: " + str(tof) + " clocks or " + str(tof * 2.6) + " ns")